# Exercise 4
Write a discrete event simulation program for a blocking system,
i.e. a system with m service units and no waiting room. The offered
traffic A is the product of the mean arrival rate and the mean
service time
## 1
The arrival process is modelled as a Poisson process. Report the
fraction of blocked customers, and a confidence interval for this
fraction. Choose the service time distribution as exponential.
Parameters: m = 10, mean service time = 8 time units, mean
time between customers = 1 time unit (corresponding to an
offered traffic of 8 Erlang), 10 x 10.000 customers.


In [1]:
import numpy as np
#import poission
import math
from scipy.stats import poisson
#import exponential
from scipy.stats import expon
import bisect

In [2]:
m = 10 #number of servers
s = 8 #mean service time
lam = 1#arrival_intensity
total_customers =10000 #10*10000
A = lam*s

In [11]:
class Customer:
    def __init__(self, arrival_time, service_time):
        self.service_time = service_time
        self.blocked = False
        
        self.event = "arrival"
        self.event_time = arrival_time
      
                
    def arrive(self, servers, event_list):
        if servers < 1:
            self.blocked = True
            return servers
        else:
            servers -= 1
            servers = max(servers, 0)
            self.event = "departure"
            self.event_time += self.service_time
            bisect.insort(event_list, self, key=lambda x: x.event_time)
            return servers
    
    def depart(self, servers):
        servers += 1
        servers = min(servers, m)
        return servers

In [12]:
#arrival time differences are exponentially distributed
arrival_interval = lambda : np.random.exponential(1/lam, size = total_customers)
service_time =lambda : expon.rvs(scale = s, size = total_customers)

In [13]:

def main_loop(arrival_interval, service_time, m, repititions = 10):
    blocked = np.zeros(repititions)
    for i in range(repititions):
        arrival_intervals = arrival_interval()
        service_times = service_time()
        arrival_times = np.cumsum(arrival_intervals)
        event_list = [Customer(arrival_times[i],service_times[i]) for i in range(len(arrival_times))]
        event_list.sort(key=lambda x: x.event_time)
        open_servers = m
        while event_list:
            event = event_list.pop(0)
            if event.event == "arrival":
                open_servers = event.arrive(open_servers, event_list)
                blocked[i] += event.blocked
            elif event.event == "departure":
                open_servers = event.depart(open_servers)
    return blocked

blocked_1 = main_loop(arrival_interval, service_time, m)

In [14]:
print("Blocking probability: ", blocked_1/total_customers)

Blocking probability:  [0.1175 0.1232 0.1265 0.1243 0.1225 0.1124 0.124  0.1241 0.1185 0.118 ]


In [15]:
#Erlang B formula
def erlang_b(m, A):
    return (A**m/math.factorial(m))/np.sum([A**i/math.factorial(i) for i in range(m+1)])

In [16]:
#Theoretical blocking probability
print(erlang_b(m, A))

0.12166106425295149


## 2
The arrival process is modelled as a renewal process using the
same parameters as in Part 1 when possible. Report the
fraction of blocked customers, and a confidence interval for this
fraction for at least the following two cases

In [17]:
# (a) Experiment with Erlang distributed inter arrival times The
#Erlang distribution should have a mean of 1
arrival_interval = lambda : np.random.gamma(m, 1/m, size = total_customers)
service_time = lambda : expon.rvs(scale = s, size = total_customers)
blocked_erlang = main_loop(arrival_interval, service_time, m)
print("Blocking probability: ", blocked_erlang/total_customers)

Blocking probability:  [0.0686 0.071  0.0644 0.0671 0.071  0.0711 0.0663 0.0713 0.0708 0.0619]


In [19]:
# hyper exponential inter arrival times. The parameters for
#the hyper exponential distribution should be
p1 = 0.8
λ1 = 0.8333
p2 = 0.2
λ2 = 5.0
arrival_interval = lambda : np.random.choice([expon.rvs(scale = 1/λ1), expon.rvs(scale = 1/λ2)], total_customers, p=[p1, p2])

service_time = lambda : expon.rvs(scale = s, size = total_customers)

blocked_hyperexp = main_loop(arrival_interval,service_time, m)
print("Blocking probability: ", blocked_hyperexp/total_customers)


Blocking probability:  [0.     0.0716 0.6379 0.7971 0.421  0.6187 0.3936 0.0014 0.7009 0.    ]


## 3
The arrival process is again a Poisson process like in Part 1.
Experiment with different service time distributions with the
same mean service time and m as in Part 1 and Part 2

In [22]:
# a) Constant service time
arrival_interval = lambda : np.random.exponential(1/lam, size = total_customers)
service_time = lambda : s*np.ones(total_customers)

blocked_constant = main_loop(arrival_interval,service_time, m)
print("Blocking probability: ", blocked_constant/total_customers)

Blocking probability:  [0.1227 0.1208 0.1208 0.1242 0.1227 0.1197 0.1259 0.1197 0.1317 0.1243]


In [24]:
# Pareto distributed service times with at least k = 1.05 and
#k = 2.05.

k = 1.05 
service_time = lambda : np.random.pareto(k, total_customers)
blocked_pareto_1 = main_loop(arrival_interval, service_time, m)
print("Blocking probability for k= 1.05: ", blocked_pareto_1/total_customers)
k = 2.05
service_time = lambda : np.random.pareto(k, total_customers)
blocked_pareto_2 = main_loop(arrival_interval, service_time, m)
print("Blocking probability for k= 2.05: ", blocked_pareto_2/total_customers)

Blocking probability for k= 1.05:  [0.0338 0.0439 0.0501 0.0592 0.0396 0.1177 0.0548 0.0423 0.0196 0.1138]
Blocking probability for k= 2.05:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [28]:
#absolute gaussian distributed service times with mean s and standard deviation s/4 #99%+ of the values are positive
service_time = lambda : np.random.normal(s, s/4, size = total_customers)
blocked_gauss = main_loop(arrival_interval, service_time, m)
print("Blocking probability: ", blocked_gauss/total_customers)

Blocking probability:  [0.1269 0.1274 0.1201 0.1214 0.1138 0.1175 0.1311 0.1189 0.1312 0.1201]


## 4
Compare confidence intervals for Parts 1, 2, and 3 then
interpret and explain differences if any.

In [ ]:
def confidence_intervals(samples):
    emp_mean = np.mean(samples)
    emp_std = np.std(samples)
    t = 1.96
    return (emp_mean - t*emp_std/np.sqrt(len(samples)), emp_mean + t*emp_std/np.sqrt(len(samples)))

#show confidence intervals for all the experiments
print("Confidence intervals for blocking probability")
print("Part 1: ", confidence_intervals(blocked_1))